You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Future of Artificial Intelligence: Trends, Key Players, and News

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Mention of the latest trends, key players, and noteworthy news in the AI industry

Key Points:
1. Latest Trends in Artificial Intelligence
- Machine learning and deep learning advancements
- Natural language processing developments
- AI ethics and bias considerations
- AI-powered automation in business processes

2. Key Players in t

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Trends, Key Players, and News

**Introduction:**

Artificial Intelligence (AI) has transformed various industries, empowering machines to execute tasks that traditionally required human intelligence. From healthcare to finance, AI technologies have been driving innovation, enhancing efficiency, and boosting productivity. The ongoing evolution of AI is influenced by the latest trends, key players in the industry, and noteworthy news, all of which contribute to shaping the future of this dynamic field.

**Latest Trends in Artificial Intelligence:**

Advancements in machine learning and deep learning have significantly expanded the capabilities of AI, allowing algorithms to learn from data and make precise predictions. The progress in natural language processing has facilitated better communication between humans and machines, leading to the widespread adoption of virtual assistants and chatbots. Nonethe

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Key Players, and News

**Introduction:**

Artificial Intelligence (AI) has transformed various industries, empowering machines to execute tasks that traditionally required human intelligence. From healthcare to finance, AI technologies have been driving innovation, enhancing efficiency, and boosting productivity. The ongoing evolution of AI is influenced by the latest trends, key players in the industry, and noteworthy news, all of which contribute to shaping the future of this dynamic field.

**Latest Trends in Artificial Intelligence:**

Advancements in machine learning and deep learning have significantly expanded the capabilities of AI, allowing algorithms to learn from data and make precise predictions. The progress in natural language processing has facilitated better communication between humans and machines, leading to the widespread adoption of virtual assistants and chatbots. Nonetheless, industry experts are actively addressing ethical concerns and biases present in AI algorithms. Moreover, the integration of AI-powered automation in business processes has revolutionized organizational operations, streamlining workflows and enhancing operational efficiency.

**Key Players in the Artificial Intelligence Industry:**

Leading the AI industry are prominent players such as Google DeepMind, renowned for its groundbreaking AI research and development. IBM Watson has been a trailblazer in cognitive computing, providing AI solutions for businesses across diverse sectors. Amazon Web Services (AWS) and Microsoft Azure offer cloud-based AI services, allowing organizations to harness AI capabilities without substantial infrastructure investments. Nvidia is recognized for its GPU technology that accelerates AI and deep learning applications, while OpenAI focuses on advancing AI research in a collaborative and ethical manner.

**Noteworthy News in Artificial Intelligence:**

Recent breakthroughs in AI research have propelled advancements in computer vision, speech recognition, and autonomous systems. Collaborations and partnerships within the industry have played a pivotal role in fostering innovation and expediting the adoption of AI technologies. Government regulations and policies concerning AI are continuously evolving to address issues related to data privacy, security, and transparency. The impact of AI on the job market and society remains a debatable topic, with experts exploring the potential consequences of automation and AI-driven decision-making on employment and societal norms.

**Conclusion:**

As AI continues to revolutionize industries and daily life, staying abreast of the latest developments is essential for individuals interested in the field. By delving into the current trends, key players, and noteworthy news in the AI industry, readers can glean valuable insights into the present state of AI and its future prospects. Subscribe to our newsletter or follow our blog for regular updates on Artificial Intelligence to remain informed and engaged in the AI revolution.

Our aim is to provide a comprehensive overview of the ever-evolving landscape of AI, catering to the interests and concerns of our target audience. Through engaging content and strategic SEO integration, our blog post on Artificial Intelligence is crafted to educate, inspire, and empower individuals intrigued by the boundless possibilities of AI technology.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Top 10 Advances in Artificial Intelligence in 2024"   #"YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Top 10 Advances in Artificial Intelligence in 2024.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Title: Top 10 Advances in Artificial Intelligence in 2024

Outline:
I. Introduction
    A. Brief overview of the rapid advancements in AI technology in 2024
II. Key Trends in AI for 2024
    A. Quantum computing integration
    B. Enhanced natural language processing
    C. Autonomous vehicles advancements
    D. Healthcare innovations
    E. Cybersecurity developments
    F. Robotics and automation improvements
    G. AI ethics and regulations
III. Key Players in AI Industry
    A.

I now can give a great answer

Final Answer:
# Top 10 Advances in Artificial Intelligence in 2024

## Introduction
Artificial Intelligence (AI) has been rapidly evolving, pushing boundaries and redefining possibilities in 2024. The advancements in AI technology have been groundbreaking, with key trends shaping the landscape of the industry. From quantum computing integration to enhanced natural language processing, the progress in AI has been remarkable.

## Key Trends in AI for 2024
One of the most significant advancements in AI in 2024 is the integration of quantum computing. This has revolutionized the capabilities of AI systems, allowing for faster and more complex calculations that were previously unimaginable. AI systems have seen significant improvements in natural language processing, enabling more accurate and nuanced interactions with users. This has paved the way for more sophisticated AI-powered applications in various industries.

The development of AI-powered autonomous v

In [14]:
Markdown(result)

# Top 10 Advances in Artificial Intelligence in 2024

## Introduction
Artificial Intelligence (AI) has been rapidly evolving, pushing boundaries and redefining possibilities in 2024. The advancements in AI technology have been groundbreaking, with key trends shaping the landscape of the industry. From quantum computing integration to enhanced natural language processing, the progress in AI has been remarkable.

## Key Trends in AI for 2024
One of the most significant advancements in AI in 2024 is the integration of quantum computing. This has revolutionized the capabilities of AI systems, allowing for faster and more complex calculations that were previously unimaginable. AI systems have seen significant improvements in natural language processing, enabling more accurate and nuanced interactions with users. This has paved the way for more sophisticated AI-powered applications in various industries.

The development of AI-powered autonomous vehicles has reached new heights in 2024, with increased safety features and improved decision-making capabilities. This has the potential to transform the transportation industry in the coming years. AI has played a crucial role in driving innovations in the healthcare sector, from personalized treatment plans to early disease detection. The integration of AI in healthcare has the potential to revolutionize patient care and improve outcomes.

With the growing threat of cyber attacks, AI-powered cybersecurity solutions have become essential in safeguarding sensitive data and networks. The advancements in AI technology have enhanced the ability to detect and prevent cyber threats in real-time. AI-powered robotics and automation have seen significant improvements in 2024, with robots becoming more agile and adaptable to various tasks. This has led to increased efficiency and productivity in industries ranging from manufacturing to logistics.

As AI technology continues to advance, the importance of ethics and regulations surrounding its use has become paramount. Companies and policymakers are grappling with the ethical implications of AI, such as bias in algorithms and data privacy concerns.

## Key Players in AI Industry
The AI industry is dominated by key players such as Google, IBM, Microsoft, Amazon, Tesla, NVIDIA, and OpenAI. These companies are at the forefront of AI innovation, driving advancements in technology and shaping the future of AI applications.

## Noteworthy News in AI for 2024
AI has revolutionized drug discovery processes, leading to the development of new treatments and therapies for various diseases. The use of AI algorithms has accelerated the drug discovery timeline and improved success rates in clinical trials. E-commerce companies are leveraging AI technology to provide personalized shopping experiences for customers. AI algorithms analyze customer data to recommend products, personalize marketing campaigns, and enhance the overall shopping experience. AI technology is being used to address climate change challenges, from optimizing energy consumption to predicting natural disasters. AI-powered climate change solutions are crucial in mitigating the impact of environmental issues and promoting sustainability.

## Target Audience Analysis
Tech enthusiasts, business professionals looking to implement AI solutions, and researchers and academics in the field of artificial intelligence are the primary audience for AI advancements in 2024. Staying updated on the latest trends and developments in AI is essential for making informed decisions and staying competitive in the industry.

## Conclusion
The top 10 advances in artificial intelligence in 2024 are shaping the future of AI technology and its applications across various industries. From quantum computing integration to healthcare innovations, AI is driving unprecedented progress and innovation. By staying informed and engaged with the latest trends in AI, individuals and businesses can harness the power of AI to drive growth and transformation. Stay updated on the latest AI advancements by subscribing to our newsletter for monthly insights and analysis.

In conclusion, the advancements in AI in 2024 are paving the way for a future where AI-powered solutions are commonplace and transformative in our daily lives. The key trends, key players, and noteworthy news in AI are indicative of the rapid progress and innovation in the industry. As we navigate the complexities and possibilities of AI technology, staying informed and engaged with the latest developments is crucial for unlocking the full potential of AI in our society.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).